# Спрос на рынке жилья (за счет ипотеки)

In [178]:
from IPython.display import Markdown as md
import pandas as pd
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy.integrate import quad
from scipy.stats import rv_continuous, norm

%run ../COMMON/common.ipynb # загрузка общих функций и констант, все, что оттуда, должно иметь префикс common.

conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strYearDBPath)) # connection к рабочей базе данных
conWorkEx = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strExYearDBPath)) # connection к к рабочей базе данных экзогенных переменных
conWorkExH=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strExParamDBPath)) # connection к рабочей базе данных экзогенных параметров
conWorkSvod=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strSvodDBPath)) # connection к рабочей базе данных СВОД


In [179]:
pd.set_option('display.max_columns',None)

## Установка временных точек

In [180]:
iFirstFactYear=2005
iLastFactYear=2019

iFirstForecastYear=iLastFactYear+1
iLastForecastYear=2030

## Загрузка исходных данных

Исходные данные:
  - Фактические значения - из базы данных `year.sqlite3`;
  - Экзогенные - из базы данных `exog_year.sqlite3`
  - Задаваемые вручную - из базы данных `exog_param.sqlite3`
  - Результаты других моделей - из базы данных `svod.sqlite3`. *В эту же базу записываем результаты данного блокнота*. А даном блокноте используются результаты блока **"Цены и себестоимость"**

In [181]:
class demand_ipot:
    pdfWork=None
    
    lstYearCodes=['ZPN_I','ZPR_I','Unmpl_s','Inc_x','MEPop','DispPop','GDP_x','CPIAv','KeyRate',
                  'HHAv','Pop_x','p_MortgLifeAv_x','p_ProbDef','ConstrPriceIZD_x',
                  'loans_and_ref_vol_MKD_vtor','ipot_debt','ipot_debt_prava','loans_and_ref_vol_MKD',
                  'sdelkiddy_ipot_x','sdelkikp_ipot_x','loans_vol','loans_vol_IZD','loans_vol_MKD',
                  'loans_vol_MKD_perv','refinance_vol','ICB','loan_rate','outstanding_debt_s',
                  'loan_vol_av','UZUipot_x','ZhilFondunits','ZhilFondIZDunits','CPR','Activ_x'] # список экзогенных и фактических данных
 
    lstSvod=['price1mddy_alt_x','price1mall_alt_x','price1mall_x','ZhilFond','ZhilFondIZD','VvodyMKDunits',
             'AvSqIZD','VvodyIZD','AddIZD_x','AvSqVv','AvSqDdy','AvSqVtor'] # результаты из блока "Цены и себестоимость"

    def __init__(self):
        assert False, 'you can\'t create variables of demand_om class!'
    
    def MakeWorkFrame():
        ''' Загрузка исходных данных
            Исходные данные:
                Фактические значения - из базы данных year.sqlite3;
                Экзогенные - из базы данных exog_year.sqlite3
                Задаваемые вручную - из базы данных exog_param.sqlite3
                Результаты других моделей - из базы данных svod.sqlite3. 
        '''
        
        pdfAct=(pd.read_sql(common.make_SELECT_YEAR_string(demand_ipot.lstYearCodes), con=conWork)
        .pipe(common.make_frame)
        .pipe(common.scale, list_fields=['KeyRate','loan_rate'], multiplier=1e2)
        .pipe(common.scale, list_fields=['Inc_x'], multiplier=1e3)
        .pipe(common.scale, list_fields=['Activ_x'], multiplier=1e-3)
        .pipe(common.scale, list_fields=['sdelkiddy_ipot_x','loans_vol_MKD_perv','refinance_vol'], multiplier=1e-6))
        
        pdfExog=(pd.read_sql(common.make_SELECT_YEAR_string(demand_ipot.lstYearCodes), con=conWorkEx)
        .pipe(common.make_frame)
        .pipe(common.scale, list_fields=['GDP_x'], multiplier=1e-3))
        
        PdfExogHandle=(pd.read_sql(common.make_SELECT_YEAR_string(demand_ipot.lstYearCodes), con=conWorkExH)
        .pipe(common.make_frame))

        pdfSvod=(pd.read_sql(common.make_SELECT_YEAR_string(demand_ipot.lstSvod), con=conWorkSvod)
        .pipe(common.make_frame)
        .pipe(common.scale, list_fields=['price1mddy_alt_x', 'price1mall_alt_x'], multiplier=1e3))

        demand_ipot.pdfWork=(pdfAct.combine_first(pdfExog).combine_first(PdfExogHandle).combine_first(pdfSvod)
        .pipe(common.scale, list_fields=['ZPR_I','Unmpl_s'], multiplier=1e2)
        .pipe(common.add, list_fields=['ZPR_I'], param=-100)
        .pipe(common.scale, list_fields=['Pop_x'], multiplier=1e3))
        
        demand_ipot.pdfWork['Inc_agg_x'] =  demand_ipot.pdfWork['Inc_x']*demand_ipot.pdfWork['Pop_x']*12/1000
        
        return demand_ipot.pdfWork
    
# a=demand_om()


In [182]:
demand_ipot.MakeWorkFrame()

,Activ_x,AvSqDdy,AvSqIZD,AvSqVtor,AvSqVv,CPIAv,CPR,ConstrPriceIZD_x,DispPop,GDP_x,HHAv,Inc_x,KeyRate,MEPop,Pop_x,UZUipot_x,Unmpl_s,VvodyIZD,VvodyMKDunits,ZPN_I,ZPR_I,ZhilFond,ZhilFondIZD,ZhilFondIZDunits,ZhilFondunits,ipot_debt,ipot_debt_prava,loan_rate,loan_vol_av,loans_and_ref_vol_MKD,loans_and_ref_vol_MKD_vtor,loans_vol_MKD_perv,outstanding_debt_s,p_MortgLifeAv_x,p_ProbDef,price1mall_alt_x,price1mall_x,price1mddy_alt_x,refinance_vol,sdelkiddy_ipot_x,sdelkikp_ipot_x,Inc_agg_x
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004,7.137613,NaN,NaN,NaN,NaN,1.108812,NaN,NaN,NaN,17.027191,2.677054,6410.300000,13.520000,NaN,144067.300,NaN,7.714864,NaN,NaN,1.225698,10.6000,NaN,NaN,15.725665,56.854367,0.015000,NaN,NaN,NaN,0.018500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.108218e+07
2005,9.750847,65.703978,138.031496,NaN,67.268041,1.127055,NaN,NaN,0.756100,21.609765,2.661470,8088.300000,12.980000,8.711611,143518.814,NaN,7.153243,17.530000,0.388000,1.269288,12.6000,2955.700000,NaN,15.670000,57.402292,0.052800,NaN,14.900000,NaN,0.056341,NaN,NaN,0.210113,NaN,NaN,21048.567265,22.245801,18491.692819,NaN,0.032140,0.109063,1.392988e+07
2006,14.046610,65.107090,138.672222,NaN,65.806452,1.096830,NaN,NaN,0.769907,26.917201,2.645886,10154.800000,8.894216,8.928732,143049.637,NaN,7.062219,19.968800,0.465000,1.243016,13.3000,3003.000000,NaN,NaN,NaN,0.233897,NaN,13.700000,NaN,0.263561,NaN,NaN,0.042982,NaN,NaN,28597.165335,31.087628,24937.896936,NaN,0.043254,0.146776,1.743169e+07
2007,20.125188,64.216907,138.458421,NaN,65.601504,1.089881,NaN,NaN,0.785663,33.247513,2.630302,12540.200000,7.512557,9.129498,142805.114,NaN,6.029393,26.307100,0.532000,1.278258,17.2000,3060.000000,NaN,NaN,NaN,0.611222,NaN,12.600000,NaN,0.556489,NaN,NaN,0.272760,NaN,0.10,37938.493305,44.119095,32765.902784,NaN,0.058210,0.197530,2.148966e+07
2008,28.022340,63.243902,136.835000,NaN,64.612676,1.141048,0.111271,NaN,0.782878,41.276849,2.614717,14863.600000,8.129107,9.301331,142742.366,0.684561,6.241192,27.367000,0.568000,1.271978,11.5000,3116.000000,NaN,NaN,NaN,1.070329,0.094510,12.900000,1.876407,0.655808,534808.0,0.121000,1.673344,17.941667,0.10,44576.706072,53.253399,37788.915681,0.000000,0.059157,0.285017,2.545999e+07
2009,29.430000,61.675214,135.934000,50.877922,63.821138,1.116977,0.110141,NaN,0.783533,38.807219,2.599133,16895.000000,11.310000,9.427587,142785.348,0.957832,8.252236,28.546140,0.492000,1.077964,-3.5000,3176.600000,974.270500,16.261078,59.547624,1.010889,0.109001,14.300000,1.172318,0.152501,136501.0,0.016000,4.701409,16.458333,0.10,42565.501231,51.857276,36473.861415,0.000000,0.012520,0.116437,2.894830e+07
2010,33.804600,60.086697,132.907292,50.985166,62.666667,1.068508,0.106253,NaN,0.782280,46.308541,2.583549,18958.400000,8.030000,9.546797,142849.468,1.340190,7.344543,25.518200,0.525000,1.124155,5.2000,3229.000000,1000.360000,16.415168,60.126706,1.129400,0.109108,13.100000,1.260847,0.380061,341061.0,0.039000,5.699277,16.358333,0.10,41870.584649,53.768465,35853.805771,0.000313,0.028376,0.270253,3.249837e+07
2011,41.627500,58.132834,133.139303,51.084428,60.683761,1.084634,0.122242,NaN,0.775441,60.114001,2.544156,20780.000000,8.120000,9.644999,142960.908,1.875183,6.495643,26.761000,0.585000,1.115359,2.8000,3286.153000,1027.764400,16.595561,60.804505,1.478980,0.077474,11.900000,1.369306,0.716944,608944.0,0.108000,3.857936,14.908333,0.10,44815.617500,44.815618,37897.472700,0.000841,0.072355,0.444096,3.564873e+07
2012,49.509600,56.425400,134.490995,51.185577,59.489633,1.050667,0.114089,NaN,0.781314,68.103450,2.504763,23221.100000,8.070000,9.747765,143201.721,2.410175,5.458664,28.377600,0.627000,1.139501,8.4000,3348.900000,1057.785000,16.777176,61.538125,1.997200,0.081758,12.290000,1.491913,1.031992,825992.0,0.206000,2.420924,14.958333,0.10,50629.950501,50.430455,41865.338092,0.006146,0.126668,0.549527,3.990362e+07


#### модель №2 "Доля просроченной задолженности по ипотеке"

In [183]:
pdfXY2 = demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,['outstanding_debt_s','Unmpl_s','ZPR_I']].dropna()
resDEBTX = smf.ols(formula='outstanding_debt_s ~ outstanding_debt_s.shift(1) + Unmpl_s + ZPR_I', missing='drop', 
                   data=pdfXY2.loc[2005:2018]).fit()
print(resDEBTX.summary())

                            OLS Regression Results                            
Dep. Variable:     outstanding_debt_s   R-squared:                       0.842
Model:                            OLS   Adj. R-squared:                  0.789
Method:                 Least Squares   F-statistic:                     15.95
Date:                Tue, 16 Feb 2021   Prob (F-statistic):           0.000603
Time:                        18:40:54   Log-Likelihood:                -11.899
No. Observations:                  13   AIC:                             31.80
Df Residuals:                       9   BIC:                             34.06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [184]:
for i in range (iFirstForecastYear,iLastForecastYear+1):
    demand_ipot.pdfWork.loc[i,'outstanding_debt_s']=(resDEBTX.params[0]
                                                    +resDEBTX.params[1]*demand_ipot.pdfWork.loc[i-1,'outstanding_debt_s']
                                                    +resDEBTX.params[2]*demand_ipot.pdfWork.loc[i,'Unmpl_s']
                                                    +resDEBTX.params[3]*demand_ipot.pdfWork.loc[i,'ZPR_I'])

#### модель №1 "Средняя ставка по выданным кредитам"

In [185]:
pdfXY = demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear, ['loan_rate', 'KeyRate', 'outstanding_debt_s']].dropna()
resLORAX = smf.ols(formula='loan_rate ~ KeyRate + outstanding_debt_s - 1', missing='drop', 
                   data=pdfXY.loc[2008:2019]).fit()
print(resLORAX.summary())

                                 OLS Regression Results                                
Dep. Variable:              loan_rate   R-squared (uncentered):                   0.980
Model:                            OLS   Adj. R-squared (uncentered):              0.976
Method:                 Least Squares   F-statistic:                              242.3
Date:                Tue, 16 Feb 2021   Prob (F-statistic):                    3.38e-09
Time:                        18:40:54   Log-Likelihood:                         -23.620
No. Observations:                  12   AIC:                                      51.24
Df Residuals:                      10   BIC:                                      52.21
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

In [186]:
ostatok = demand_ipot.pdfWork.loc[iLastFactYear,'loan_rate']-resLORAX.predict(demand_ipot.pdfWork).loc[iLastFactYear]
demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loan_rate']=resLORAX.predict(demand_ipot.pdfWork).loc[iFirstForecastYear:iLastForecastYear]+ostatok

#### модель №3 "Средний размер ипотечного кредита"

In [187]:
demand_ipot.pdfWork['D15']=0
demand_ipot.pdfWork.loc[2015,'D15']=1

demand_ipot.pdfWork['flat_price']=((demand_ipot.pdfWork['AvSqVv']*demand_ipot.pdfWork['price1mall_x'])/(demand_ipot.pdfWork['AvSqVv'].shift(1)*demand_ipot.pdfWork['price1mall_x'].shift(1))-1)*100
demand_ipot.pdfWork['ZPN_rI']=(demand_ipot.pdfWork['ZPN_I']-1)*100
demand_ipot.pdfWork['loan_vol_av_rI']=(demand_ipot.pdfWork['loan_vol_av']/demand_ipot.pdfWork['loan_vol_av'].shift(1)-1)*100

pdfXY3 = demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,['loan_vol_av_rI','flat_price','ZPN_rI','D15']].dropna()
resLOVORI = smf.ols(formula='loan_vol_av_rI ~ flat_price.shift(1) + ZPN_rI + D15 -1', missing='drop', 
                    data=pdfXY3.loc[2009:2016]).fit()
print(resLOVORI.summary())

                                 OLS Regression Results                                
Dep. Variable:         loan_vol_av_rI   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.968
Method:                 Least Squares   F-statistic:                              71.37
Date:                Tue, 16 Feb 2021   Prob (F-statistic):                    0.000627
Time:                        18:40:55   Log-Likelihood:                         -9.9323
No. Observations:                   7   AIC:                                      25.86
Df Residuals:                       4   BIC:                                      25.70
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

In [188]:
demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loan_vol_av_rI']=resLOVORI.predict(demand_ipot.pdfWork).loc[iFirstForecastYear:iLastForecastYear]

for i in range (iFirstForecastYear,iLastForecastYear+1):
    demand_ipot.pdfWork.loc[i,'loan_vol_av']=demand_ipot.pdfWork.loc[i-1,'loan_vol_av']*(demand_ipot.pdfWork.loc[i,'loan_vol_av_rI']/100+1)

# demand_ipot.pdfWork[['loan_vol_av','loan_vol_av_rI','flat_price','ZPN_rI','D15']]

#### модель №4 "Площадь жилья, приобретенного за счет ипотеки на первичном рынке"

In [189]:
demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'squareddy_ipot_x']=demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'AvSqDdy']*demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'sdelkiddy_ipot_x']

In [190]:
demand_ipot.pdfWork[['squareddy_ipot_x']]

,squareddy_ipot_x
date,
2004,NaN
2005,2.111717
2006,2.816116
2007,3.738091
2008,3.741290
2009,0.772200
2010,1.705002
2011,4.206192
2012,7.147313


##### расчет совокупного объема потенциальных кредитов

In [191]:
demand_ipot.pdfWork['month_pay_perv']=np.pmt(demand_ipot.pdfWork['loan_rate']/12/100,demand_ipot.pdfWork['p_MortgLifeAv_x']*12,-demand_ipot.pdfWork['price1mddy_alt_x']*54*0.7)/1000
demand_ipot.pdfWork['month_pay_all']=np.pmt(demand_ipot.pdfWork['loan_rate']/12/100,demand_ipot.pdfWork['p_MortgLifeAv_x']*12,-demand_ipot.pdfWork['price1mall_x']*1000*54*0.7)/1000
demand_ipot.pdfWork['mininc_for_loan_approval']=demand_ipot.pdfWork['month_pay_perv']*1000/0.35/demand_ipot.pdfWork['HHAv']

In [192]:
demand_ipot.pdfWork[['month_pay_perv','month_pay_all','loan_rate','p_MortgLifeAv_x','price1mddy_alt_x','mininc_for_loan_approval']]

,month_pay_perv,month_pay_all,loan_rate,p_MortgLifeAv_x,price1mddy_alt_x
date,,,,,
2004,NaN,NaN,NaN,NaN,NaN
2005,NaN,NaN,14.900000,NaN,18491.692819
2006,NaN,NaN,13.700000,NaN,24937.896936
2007,NaN,NaN,12.600000,NaN,32765.902784
2008,17.062571,24.045144,12.900000,17.941667,37788.915681
2009,18.181701,25.850115,14.300000,16.458333,36473.861415
2010,16.787391,25.175354,13.100000,16.358333,35853.805771
2011,17.138762,20.267426,11.900000,14.908333,37897.472700
2012,19.307776,23.257902,12.290000,14.958333,41865.338092


In [194]:
class kolmakov_gen(rv_continuous):
    """ Распределения доходов населения по уровню среднедушевого среднемесячного денежного дохода в денежном выражении """
        
    def _pdf(self, x, sigma, mu):
        Px = np.exp(-(np.log(x)-mu)**2 / (2*sigma**2))
        return Px / (sigma*np.sqrt(2*np.pi))
#         return lognorm.pdf(x, s=sigma, scale=np.exp(mu)) * x
    
    def _cdf(self, x, sigma, mu):
        return quad(self._pdf, 0, x, args=(sigma, mu))[0] 


kolmakov = kolmakov_gen(a=0.0, name='kolmakov', shapes='sigma, mu')

In [195]:
def _calc_perca_income(x):
    return kolmakov.cdf(x['mininc_for_loan_approval'], sigma=x['DispPop'], mu=x['MEPop'])

demand_ipot.pdfWork['_income_percapita']=demand_ipot.pdfWork[['mininc_for_loan_approval', 'DispPop', 
                                         'MEPop']].apply(_calc_perca_income, axis=1)

In [196]:
demand_ipot.pdfWork['_s_income_available_ipot']=(demand_ipot.pdfWork['Inc_agg_x']-12*demand_ipot.pdfWork['_income_percapita']*demand_ipot.pdfWork['Pop_x']/1000)/demand_ipot.pdfWork['Inc_agg_x']
demand_ipot.pdfWork['poten_cashpay_4loan']=(demand_ipot.pdfWork['Inc_agg_x']-12*demand_ipot.pdfWork['_income_percapita']*demand_ipot.pdfWork['Pop_x']/1000)*0.35
demand_ipot.pdfWork['poten_loans_vol']=-np.pv(demand_ipot.pdfWork['loan_rate']/100,demand_ipot.pdfWork['p_MortgLifeAv_x'],demand_ipot.pdfWork['poten_cashpay_4loan'])

In [197]:
def _calc_superrich_perca_income(x):
    return 1-kolmakov.cdf(x['_inc_percapita_superrich'], sigma=x['DispPop'], mu=x['MEPop'])/x['Inc_x']

demand_ipot.pdfWork['p_superrich']=0.02
demand_ipot.pdfWork['_inc_percapita_superrich']=np.exp(norm.ppf(1-demand_ipot.pdfWork['p_superrich'], 
                                                                loc=demand_ipot.pdfWork['MEPop'], 
                                                                scale=demand_ipot.pdfWork['DispPop']))

demand_ipot.pdfWork['_p_income_superr']=demand_ipot.pdfWork[['_inc_percapita_superrich', 'DispPop', 
                                        'MEPop', 'Inc_x']].apply(_calc_superrich_perca_income, axis=1)

In [198]:
demand_ipot.pdfWork[['poten_loans_vol','loan_rate','p_MortgLifeAv_x','poten_cash_pay_for_loan','_s_income_available_ipot','poten_cash_pay_for_loan','_inc_percapita_superrich','_p_income_superr']]

,poten_loans_vol,loan_rate,p_MortgLifeAv_x,poten_cash_pay_for_loan,_s_income_available_ipot,poten_cash_pay_for_loan,_inc_percapita_superrich,_p_income_superr
date,,,,,,,,
2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,NaN,14.900000,NaN,NaN,NaN,NaN,28694.187904,0.097854
2006,NaN,13.700000,NaN,NaN,NaN,NaN,36677.782253,0.100131
2007,NaN,12.600000,NaN,NaN,NaN,NaN,46307.201786,0.101093
2008,3.311209e+07,12.900000,17.941667,4.817756e+06,0.540653,4.817756e+06,54675.159452,0.100891
2009,3.593185e+07,14.300000,16.458333,5.778706e+06,0.570347,5.778706e+06,62116.497962,0.102205
2010,4.983439e+07,13.100000,16.358333,7.534001e+06,0.662363,7.534001e+06,69801.034336,0.099284
2011,5.857888e+07,11.900000,14.908333,8.575109e+06,0.687270,8.575109e+06,75929.435497,0.097049
2012,6.331853e+07,12.290000,14.958333,9.450828e+06,0.676690,9.450828e+06,85168.499213,0.101453


In [200]:
demand_ipot.pdfWork.loc[2005:2024,'excess_of_flat_cost']=1.5
for i in range (2025,2031):
    demand_ipot.pdfWork.loc[i,'excess_of_flat_cost']=demand_ipot.pdfWork.loc[i-1,'excess_of_flat_cost']-0.25
    
demand_ipot.pdfWork['poten_cashpay_4loan_exsuperrich']=demand_ipot.pdfWork['Inc_agg_x']*(demand_ipot.pdfWork['_s_income_available_ipot']-demand_ipot.pdfWork['_p_income_superr'])*0.35+(12*demand_ipot.pdfWork['mininc_for_loan_approval']*demand_ipot.pdfWork['p_superrich']*demand_ipot.pdfWork['excess_of_flat_cost']*demand_ipot.pdfWork['Pop_x'])/1000
demand_ipot.pdfWork['poten_loans_vol_exsuperrich']=-np.pv(demand_ipot.pdfWork['loan_rate']/100,demand_ipot.pdfWork['p_MortgLifeAv_x'],demand_ipot.pdfWork['poten_cashpay_4loan_exsuperrich'])

In [201]:
demand_ipot.pdfWork[['poten_loans_vol_exsuperrich','poten_cashpay_4loan_exsuperrich']]

,poten_loans_vol_exsuperrich,poten_cashpay_4loan_exsuperrich
date,,
2004,NaN,NaN
2005,NaN,NaN
2006,NaN,NaN
2007,NaN,NaN
2008,3.351796e+07,4.876809e+06
2009,3.588105e+07,5.770536e+06
2010,4.867966e+07,7.359428e+06
2011,5.707385e+07,8.354794e+06
2012,6.143239e+07,9.169304e+06
